# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [ ]:
import logging
import os
import csv
import shutil

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.widgets import RunDetails
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.core.dataset import Dataset
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice


# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [ ]:
#create an experiment for hyperdrive run
ws = Workspace.from_config()
experiment_name = 'Hyperdrive-California-housing-price-projection'

experiment=Experiment(ws, experiment_name)

In [ ]:
#create compute cluster
compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',vm_priority='lowpriority',max_nodes=4)
compute_target = ComputeTarget.create(ws, "compute-hyperdirve", compute_config)

## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [ ]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(slack_factor=0.1, evaluation_interval=1)

#TODO: Create the different params that you will be using during training
ps = RandomParameterSampling({
    "--max_depth": choice(2, 4, 8, 16),
    "--min_samples_split": choice(50, 100, 500, 1000), 
    "--min_samples_leaf": choice(10, 50, 100),
    })
#TODO: Create your estimator and hyperdrive config
estimator = SKLearn(source_directory='.', compute_target = compute_target, vm_priority = 'lowpriority', entry_script='train.py')

hyperdrive_run_config = HyperDriveConfig(
    hyperparameter_sampling=ps,
    primary_metric_name='r2_score', 
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    max_total_runs = 20,
    max_concurrent_runs = 3,
    policy= early_termination_policy,
    estimator = estimator)

In [ ]:
#TODO: Submit your experiment
from azureml.train.hyperdrive import HyperDriveRun
hyperdrive_run = experiment.submit(hyperdrive_run_config)


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion()

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [ ]:
import joblib
# Get your best run and save the model from that run.
best_hyperdrive_run = hyperdrive_run.get_best_run_by_primary_metric()
best_hyperdrive_model = best_hyperdrive_run.register_model(model_name = 'DecisionTreeregressionBest', model_path = './outputs_hyperdrive/model.joblib')

In [ ]:
#TODO: Save the best model


## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service